In [1]:
import pandas as pd
import numpy as np
from itertools import product
import plotly.graph_objs as go

In [95]:
class MeanReversionBacktester():

  def __init__(self, symbol, SMA, dev, start, end, tc):

    self.symbol = symbol
    self.SMA = SMA
    self.dev = dev
    self.start = start
    self.end = end
    self.tc = tc
    self.results = None
    self.get_data()
    self.prepare_data()

  def __repr__(self):

    return f'MeanReversionBacktester(symbol = {self.symbol}, SMA = {self.SMA}, dev = {self.dev}, start = {self.start}, end = {self.end}, tc={self.tc})'

  def get_data(self):

    if self.symbol == 'EUR/USD':

      raw = pd.read_csv(filepath_or_buffer='../../resources/intraday.csv', parse_dates=['time'], index_col='time')
      raw = raw.Close.to_frame().dropna()
      raw["returns"] = np.log(raw / raw.shift(1))
      self.data = raw

    # Maybe define an else case to get data from yahoo finance

  def prepare_data(self):

    data = self.data.copy()
    data['SMA'] = data['Close'].rolling(self.SMA).mean()
    data['Lower'] = data['SMA'] - data['Close'].rolling(self.SMA).std() * self.dev
    data['Upper'] = data['SMA'] + data['Close'].rolling(self.SMA).std() * self.dev
    data.dropna(inplace=True)
    self.data = data

  def set_parameters(self, SMA = None, dev = None):
        
    if SMA is not None:
        self.SMA = SMA
        self.data["SMA"] = self.data["Close"].rolling(self.SMA).mean()
        self.data["Lower"] = self.data["SMA"] - self.data["Close"].rolling(self.SMA).std() * self.dev
        self.data["Upper"] = self.data["SMA"] + self.data["Close"].rolling(self.SMA).std() * self.dev
        
    if dev is not None:
        self.dev = dev
        self.data["Lower"] = self.data["SMA"] - self.data["Close"].rolling(self.SMA).std() * self.dev
        self.data["Upper"] = self.data["SMA"] + self.data["Close"].rolling(self.SMA).std() * self.dev

  def test_strategy(self):

    data = self.data.copy().dropna()
    data["distance"] = data.Close - data.SMA
    data["position"] = np.where(data.Close < data.Lower, 1, np.nan)
    data["position"] = np.where(data.Close > data.Upper, -1, data["position"])
    data["position"] = np.where(data.distance * data.distance.shift(1) < 0, 0, data["position"])
    data["position"] = data.position.ffill().fillna(0)
    data["strategy"] = data.position.shift(1) * data["returns"]
    data.dropna(inplace=True)
    
    # determine the number of trades in each bar
    data["trades"] = data.position.diff().fillna(0).abs()
    
    # subtract transaction/trading costs from pre-cost return
    data['strategy_net'] = data.strategy - data.trades * self.tc
    
    data["creturns"] = data["returns"].cumsum().apply(np.exp)
    data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
    data["cstrategy_net"] = data["strategy_net"].cumsum().apply(np.exp)
    self.results = data
    
    perf = data["cstrategy"].iloc[-1] # absolute performance of the strategy
    outperf = perf - data["creturns"].iloc[-1] # out-/underperformance of strategy
    
    return round(perf, 6), round(outperf, 6)

  def plot_results(self):

    if self.results is None:
      print("Run test_strategy() first")

    else:
      fig = go.Figure()

      fig.add_trace(go.Scatter(x=self.results.index, y=self.results.creturns, name='Returns (Base)'))
      fig.add_trace(go.Scatter(x=self.results.index, y=self.results.cstrategy, name='Returns (Strategy)'))
      fig.add_trace(go.Scatter(x=self.results.index, y=self.results.cstrategy_net, name='Returns (Strategy + cost)'))

      title = f"{self.symbol} | SMA = {self.SMA} | dev = {self.dev} | TC = {self.tc}"
      fig.update_layout(title=title, xaxis_title='Time', yaxis_title='Price')

      fig.show()

  def optimize_parameters(self, SMA_range, dev_range):

    combinations = list(product(range(*SMA_range), range(*dev_range)))
    
    # test all combinations
    results = []
    for comb in combinations:
        self.set_parameters(comb[0], comb[1])
        results.append(self.test_strategy()[0])
    
    best_perf = np.max(results) # best performance
    opt = combinations[np.argmax(results)] # optimal parameters
    
    # run/set the optimal strategy
    self.set_parameters(opt[0], opt[1])
    self.test_strategy()
                
    # create a df with many results
    many_results =  pd.DataFrame(data = combinations, columns = ["SMA", "dev"])
    many_results["performance"] = results
    self.results_overview = many_results
                        
    return opt, best_perf

In [96]:
obj = MeanReversionBacktester(symbol="EUR/USD", SMA=30, dev=2, start='2018', end='2020', tc=0.00007)

In [97]:
obj

MeanReversionBacktester(symbol = EUR/USD, SMA = 30, dev = 2, start = 2018, end = 2020, tc=7e-05)

In [98]:
obj.data

,Close,returns,SMA,Lower,Upper
time,,,,,
2018-01-11 04:00:00+00:00,1.194770,-0.000498,1.200504,1.190849,1.210160
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122
...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235


In [99]:
obj.test_strategy()

(1.078136, 0.14042)

In [100]:
obj.results

,Close,returns,SMA,Lower,Upper,distance,position,strategy,trades,strategy_net,creturns,cstrategy,cstrategy_net
time,,,,,,,,,,,,,
2018-01-11 10:00:00+00:00,1.204915,0.008455,1.200628,1.190841,1.210415,0.004287,0.0,0.000000,0.0,0.000000,1.008491,1.000000,1.000000
2018-01-11 16:00:00+00:00,1.203205,-0.001420,1.200500,1.190964,1.210035,0.002705,0.0,-0.000000,0.0,-0.000000,1.007060,1.000000,1.000000
2018-01-11 22:00:00+00:00,1.205165,0.001628,1.200524,1.190943,1.210104,0.004641,0.0,0.000000,0.0,0.000000,1.008700,1.000000,1.000000
2018-01-12 04:00:00+00:00,1.212530,0.006093,1.200748,1.190374,1.211122,0.011782,-1.0,0.000000,1.0,-0.000070,1.014865,1.000000,0.999930
2018-01-12 10:00:00+00:00,1.214820,0.001887,1.201086,1.189583,1.212589,0.013734,-1.0,-0.001887,0.0,-0.001887,1.016781,0.998115,0.998045
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-29 22:00:00+00:00,1.119920,0.002092,1.111391,1.105215,1.117567,0.008529,-1.0,-0.002092,0.0,-0.002092,0.937352,1.078555,1.070207
2019-12-30 04:00:00+00:00,1.119940,0.000018,1.111553,1.104754,1.118352,0.008387,-1.0,-0.000018,0.0,-0.000018,0.937369,1.078536,1.070188
2019-12-30 10:00:00+00:00,1.120095,0.000138,1.111777,1.104318,1.119235,0.008318,-1.0,-0.000138,0.0,-0.000138,0.937498,1.078387,1.070040


In [101]:
obj.results.trades.value_counts()

0.0    1929
1.0     111
Name: trades, dtype: int64

In [102]:
obj.plot_results()

In [103]:
SMA_range = (10, 150)
dev_range = (1, 3)

In [104]:
obj.optimize_parameters(SMA_range=SMA_range, dev_range=dev_range)

((58, 1), 1.271249)

In [105]:
obj_optimized = MeanReversionBacktester(symbol="EUR/USD", SMA=58, dev=1, start='2018', end='2020', tc=0.00007)

In [106]:
obj_optimized.plot_results()

Run test_strategy() first


In [107]:
obj.test_strategy()

(1.271249, 0.371832)

In [108]:
obj.plot_results()